In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random as rd
import csv
import numpy as np
import math
from student_utils import *
from utils import *
# from ipynb.fs.full.gen_inputs import add_weights2

# Compare two methods

## Select the best output for each input

# Metric TSP Solver

## Method 1: A polynomial-time Algo with cost <= 2*optimal

1. Take the MST of G.
2. Do a DPS on MST to get PT(pseudotour).
3. Go from PT to T* (Take each vertex only the first time it appears in PT)

In [ ]:
G1 = nx.complete_graph(10)
add_weights2(G1, 1, 10)
print(list(G1.edges().data()))
plt.subplot()
nx.draw(G1, node_color='yellow', with_labels=True)

In [ ]:
ta_homes = [0,1,2,3,4]
G1_sub = G1.subgraph(ta_homes)
print(list(G1_sub.edges().data()))
plt.subplot()
nx.draw(G1_sub, node_color='yellow', with_labels=True)

In [ ]:
PT = nx.minimum_spanning_tree(G1_sub)
# print(sorted(PT.edges(data=True)))
print(PT.edges(data=True))

plt.subplot()
nx.draw(PT, node_color='yellow', with_labels=True)

In [ ]:
# TODO: implement dfs yielding pseudotour
list(nx.dfs_edges(PT, source=0))

# Rao drives every TA home: this is a TSP

## Nearest Neighbor Algorithm

In [ ]:
def solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph1, msg1 = adjacency_matrix_to_graph(adjacency_matrix)
    path = []
    drop_offs = {}
    current_idx = start_idx
    while homes_idx:
        lst_dist = [(h_idx, nx.shortest_path_length(graph1, current_idx, h_idx, 'weight')) for h_idx in homes_idx]
        idx, _ = min(lst_dist, key=lambda x: x[1])
        shortest_path = nx.shortest_path(graph1, current_idx, idx, 'weight')
        shortest_path.pop()
        path.extend(shortest_path)
        drop_offs[idx] = [idx]
        current_idx = idx
        homes_idx.remove(idx)
    last_shortest_path = nx.shortest_path(graph1, current_idx, start_idx, 'weight')
    path.extend(last_shortest_path)
    return path, drop_offs

In [ ]:
data = read_file('test_inputs/27_50.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
solve(lst_loc, lst_house, start_loc, adj_mat)

## 2-Approximation using MST

In [ ]:
#code

In [ ]:
#test

## 1.5-Approximation using Christofides Algorithm

In [ ]:
#code

In [ ]:
#test

## Branch-and-bound using the algorithm in the textbook

In [ ]:
#code

In [ ]:
#test

## Branch-and-bound using another lower-bound

In [ ]:
#code

In [ ]:
#test

# Rao drives TAs to some dropoff locations (clusters)